In [44]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import sys
from datetime import datetime
import utils
import scripts.crop_type
import scripts.geom_processing
import scripts.observation_processing
import scripts.automation
import ipywidgets as widgets
from IPython.display import display
import geopandas as gpd
import pandas as pd
import utils

from utils import crop_variations, season_codes, variation_codes, group_codes #, crop_codes, database_schema
from scripts.crop_type import map_variations, code_variations, parse_crop_type, add_crop_code
from scripts.geom_processing import check_invalid_geoms, check_repeating_geoms, check_repeating_geoms_s2cell
from scripts.observation_processing import to_datetime,to_numeric,assign_date,yield_process

#sys.path.append('..')


In [45]:
#Checking the generalizability of the data harmonization process for simple datasets (shapefile/geojson/parquet + xlsx/csv)
# We can test this out on datasets like Zelena/IMC/Kernel to get standardized outputs without needing a lot of manual interjection

#test_path_shp = "../../data/raw/UKRAINE/ZELENA_DOLYNA/2021_zelena_dolyna/2021_zelena_dolyna.shp"
#test_path_1 = list([test_path_shp])
test_path_par = "data/2023_kernel.parquet"
test_path_xlsx = "data/2023_kernel_table.xlsx"


In [46]:
#test_path_par = "../../data/raw/UKRAINE/KERNEL/2023_kernel/2023_kernel.parquet"
#test_path_xlsx = "../../data/raw/UKRAINE/KERNEL/2023_kernel/2023_kernel_table.xlsx"

test_path_2 = list([test_path_par,test_path_xlsx])

#Merges the geospatial and tabular files (if required)
dataset =scripts.automation.read_ground_data(file_paths=test_path_2, join_key=('Field','Field'))
#We work with yearly data 
dataset = dataset[dataset['Year'] == 2023]
dataset.head()


--- DEBUGGING EXCEL LOAD: 2023_kernel_table.xlsx ---
Columns detected: ['Unnamed: 0', 'Year', 'Crop', 'Region', 'Yield', 'Yield_clean', 'Harvest_area', 'Field_area', 'Planned_area', 'Sowing', 'Harvesting', 'N', 'P', 'K', 'Field']
Row count: 4195
First 2 rows:
   Unnamed: 0  Year   Crop               Region Yield Yield_clean  \
0           0  2022  Herbs  Кременчуцький район   NaN         NaN   
1           1  2022  Herbs      Оржицький район   NaN         NaN   

   Harvest_area  Field_area  Planned_area Sowing Harvesting          N  \
0         120.9       122.6         122.6    NaT        NaT  11181.120   
1         187.6        93.9          93.9    NaT        NaT  11222.442   

        P      K  Field  
0    0.00    0.0      0  
1  479.64  831.0      1  
Sanitizing keys: Casting 'Field' and 'Field'...
Merging on Field == Field


,geometry,Field,Unnamed: 0,Year,Crop,Region,Yield,Yield_clean,Harvest_area,Field_area,Planned_area,Sowing,Harvesting,N,P,K
1,"POLYGON Z ((33.20118 49.19568 -100000, 33.2011...",0,547,2023,Herbs,Кременчуцький район,NaN,NaN,NaN,122.60,120.90,NaT,NaT,11297.60,0.0,0.0
3,"POLYGON Z ((32.61933 49.99669 -100000, 32.6191...",1,548,2023,Herbs,Оржицький район,NaN,NaN,NaN,93.90,93.90,NaT,NaT,NaN,NaN,NaN
5,"POLYGON Z ((32.17066 50.28364 -100000, 32.1706...",10,555,2023,Herbs,Пирятинський район,NaN,NaN,NaN,84.09,84.09,2023-04-17,NaT,7406.00,0.0,0.0
7,"MULTIPOLYGON Z (((32.9605 50.31382 -100000, 32...",1000,1132,2023,Soy,Чорнухинський район,NaN,NaN,NaN,160.31,160.77,2023-05-14,NaT,5554.80,1028.8,257.2
8,"POLYGON Z ((32.96351 50.30416 -100000, 32.9631...",1001,1133,2023,Soy,Чорнухинський район,NaN,NaN,NaN,34.03,34.03,2023-05-12,NaT,1161.98,217.6,54.4


In [47]:
#====== GEOMETRIC OPERATIONS================

#Project to EPSG:4326
dataset_4326 = dataset.to_crs(4326)

#Check invalid or null geoms 
dataset_check_valid_geoms = scripts.geom_processing.check_invalid_geoms(gdf = dataset_4326)
if dataset_check_valid_geoms[0].empty:
    print("no invalid geoms")
else:
    print("The invalid geoms are:",dataset_check_valid_geoms[0])

#Check duplicate geoms    
dataset_4326_check_duplicate_geoms = scripts.geom_processing.check_repeating_geoms_s2cell(gdf = dataset_check_valid_geoms[1], col_name = "Geom_id", granularity = 25)
print("Duplicate geoms: ",len(dataset_4326_check_duplicate_geoms[0]))
print("Dataset with Geom_ids: ",len(dataset_4326_check_duplicate_geoms[1]))

#Remove 3D geoms
dataset_4326_non_duplicate_geoms = dataset_4326_check_duplicate_geoms[1]
dataset_4326_non_duplicate_geoms["geometry"] = dataset_4326_non_duplicate_geoms["geometry"].apply(scripts.geom_processing.geometry_remove_z)
dataset_4326_non_duplicate_geoms.columns

no invalid geoms
Geom ids assigned to the geodataframe


Duplicate geoms:  0
Dataset with Geom_ids:  2134


c:\Users\gmana\Desktop\GROUND DATA AUTOMATION\scripts\geom_processing.py:88: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = gdf.geometry.centroid


Index(['geometry', 'Field', 'Unnamed: 0', 'Year', 'Crop', 'Region', 'Yield',
       'Yield_clean', 'Harvest_area', 'Field_area', 'Planned_area', 'Sowing',
       'Harvesting', 'N', 'P', 'K', 'lat', 'lon', 'Geom_id'],
      dtype='object')

In [48]:
#======CROP TYPE OPERATIONS================
ct_column = 'Crop'
#Wrapper function removes all the null crop type columns and assigns crop, group and season to each row
dataset_4326_non_duplicate_geoms = scripts.crop_type.remove_null_ct(dataset=dataset_4326_non_duplicate_geoms,ct_column_name=ct_column)

data_crop = scripts.automation.crop_type_preprocess(dataset=dataset_4326_non_duplicate_geoms, crop_column = ct_column)
dataset_4326_non_duplicate_geoms = dataset_4326_non_duplicate_geoms.drop(columns=[ct_column])
data_crop.head()

Number of null crop rows removed from the dataset:  0
Number of null crop rows removed from the dataset:  0
The crop types in this dataset:  Crop
Sunflower    607
Soybean      419
Maize        363
Wheat        336
Rapeseed     269
Fallow       114
Grassland     20
Barley         6
Name: count, dtype: int64
The number of variations in this dataset:  Series([], Name: count, dtype: int64)
The number of group in this dataset:  Crop_Group
Oilseeds          876
Cereals           705
Legumes/Pulses    419
Non Crop          134
Name: count, dtype: int64


,Crop_Group,Crop_Season,Crop_Variation,Crop
0,Non Crop,None,None,Grassland
1,Non Crop,None,None,Grassland
2,Non Crop,None,None,Grassland
3,Legumes/Pulses,None,None,Soybean
4,Legumes/Pulses,None,None,Soybean


In [49]:
#======YIELD OPERATIONS================

yield_col = "Yield" #Assign yield column name

#Convert the yield col to numeric
dataset_4326_non_duplicate_geoms[yield_col] = scripts.observation_processing.to_numeric(series = dataset_4326_non_duplicate_geoms[yield_col])

#Convert the yield col units to ton/ha (db standard)
dataset_4326_non_duplicate_geoms[yield_col] = scripts.observation_processing.yield_process(column = dataset_4326_non_duplicate_geoms[yield_col],current_unit = "ton/ha")

The yield units have been successfully changed from ton/ha to ton/ha 



In [50]:
#Datetime columns standardization
cols_to_convert = {
    "Sowing": {"new_col": "Sow_Date", "format": "%Y-%m-%d"},
    #"Harvesting": {"new_col": "Harvest_Date", "format": "%Y-%m-%d"}  
}
dataset_4326_non_duplicate_geoms = scripts.observation_processing.convert_multiple_datetime(df = dataset_4326_non_duplicate_geoms, dict_mapping = cols_to_convert,func = scripts.observation_processing.to_datetime)
#dataset_4326_non_duplicate_geoms["Sow_Date"].unique()

In [51]:
#Assign provider name and Src_file
provider_name = "Kernel" #Assign provider name
path_name = test_path_par + " + " + test_path_xlsx #Assign path to original file
  
dataset_4326_non_duplicate_geoms["Provider"] = provider_name
dataset_4326_non_duplicate_geoms["Src_File"] =  path_name 

#Join all the outputs 
data_refined_ct = pd.concat([data_crop,dataset_4326_non_duplicate_geoms],axis=1)

#Generate Sow month, year and day
data_refined_ct["Sow_Year"] = data_refined_ct["Sow_Date"].dt.year.astype("Int64")
data_refined_ct["Sow_Month"] = data_refined_ct["Sow_Date"].dt.month.astype("Int64")
data_refined_ct["Sow_Day"] = data_refined_ct["Sow_Date"].dt.day.astype("Int64")
#data_refined_ct["Sow_Day"].unique()

In [52]:
#FOR UKRAINE
#Assign values to Est_Sow_Year for different cropping season 
est_year_data_2023 = 2023
est_year_data_2023_winter = est_year_data_2023 - 1 

#Assigning Est_Sow_Year values 
mask_null_sow_year = data_refined_ct["Sow_Year"].isnull()
# Assign est_year_data_2023_winter where Season == "Winter"
data_refined_ct.loc[
    mask_null_sow_year & (data_refined_ct["Crop_Season"] == "Winter"),
    "Est_Sow_Year"
] = est_year_data_2023
# Assign est_year_data_2023 where Season != "Winter"
data_refined_ct.loc[
    mask_null_sow_year & (data_refined_ct["Crop_Season"] != "Winter"),
    "Est_Sow_Year"
] = est_year_data_2023_winter
data_refined_ct.head()

,Crop_Group,Crop_Season,Crop_Variation,Crop,geometry,Field,Unnamed: 0,Year,Region,Yield,...,lat,lon,Geom_id,Sow_Date,Provider,Src_File,Sow_Year,Sow_Month,Sow_Day,Est_Sow_Year
0,Non Crop,None,None,Grassland,"POLYGON ((33.20118 49.19568, 33.20114 49.19569...",0,547,2023,Кременчуцький район,NaN,...,49.187486,33.207598,40d74a3f67010c,NaT,Kernel,data/2023_kernel.parquet + data/2023_kernel_ta...,<NA>,<NA>,<NA>,2022.0
1,Non Crop,None,None,Grassland,"POLYGON ((32.61933 49.99669, 32.61915 49.99669...",1,548,2023,Оржицький район,NaN,...,49.992112,32.613038,40d6888749dff4,NaT,Kernel,data/2023_kernel.parquet + data/2023_kernel_ta...,<NA>,<NA>,<NA>,2022.0
2,Non Crop,None,None,Grassland,"POLYGON ((32.17066 50.28364, 32.17068 50.28369...",10,555,2023,Пирятинський район,NaN,...,50.286262,32.164984,40d5c94c4f5fbc,2023-04-17,Kernel,data/2023_kernel.parquet + data/2023_kernel_ta...,2023,4,17,NaN
3,Legumes/Pulses,None,None,Soybean,"MULTIPOLYGON (((32.9605 50.31382, 32.9603 50.3...",1000,1132,2023,Чорнухинський район,NaN,...,50.309205,32.956950,40d60d3023aa1c,2023-05-14,Kernel,data/2023_kernel.parquet + data/2023_kernel_ta...,2023,5,14,NaN
4,Legumes/Pulses,None,None,Soybean,"POLYGON ((32.96351 50.30416, 32.96315 50.30419...",1001,1133,2023,Чорнухинський район,NaN,...,50.301043,32.963876,40d60d359155cc,2023-05-12,Kernel,data/2023_kernel.parquet + data/2023_kernel_ta...,2023,5,12,NaN


In [53]:
#Assign season id to all the rows 
data_final_refined = scripts.observation_processing.assign_season_id(data_refined_ct)
data_final_refined.columns

Index(['Crop_Group', 'Crop_Season', 'Crop_Variation', 'Crop', 'geometry',
       'Field', 'Unnamed: 0', 'Year', 'Region', 'Yield', 'Yield_clean',
       'Harvest_area', 'Field_area', 'Planned_area', 'Sowing', 'Harvesting',
       'N', 'P', 'K', 'lat', 'lon', 'Geom_id', 'Sow_Date', 'Provider',
       'Src_File', 'Sow_Year', 'Sow_Month', 'Sow_Day', 'Est_Sow_Year',
       'Season_id'],
      dtype='object')

In [55]:
#Assign final standardized columns names 
final_cols = scripts.automation.display_final_cols(df = data_final_refined, schema_dict = utils.target_schema_grouped)
final_cols.show()

In [36]:
rename_dict = final_cols.get_mapping()
rename_dict

{'Crop': 'Crop',
 'Season_id': 'Season_id',
 'Geom_id': 'Geom_id',
 'Provider': 'Provider',
 'Field': 'Src_id',
 'Src_File': 'Src_File',
 'Crop_Group': 'Crop_Group',
 'Crop_Season': 'Crop_Season',
 'Crop_Variation': 'Crop_Variation',
 'geometry': 'geometry',
 'Sow_Date': 'Sow_Date',
 'Est_Sow_Year': 'Est_Sow_Year',
 'Sow_Day': 'Sow_Day',
 'Sow_Month': 'Sow_Month',
 'Sow_Year': 'Sow_Year',
 'Yield': 'Yield'}

In [37]:
data_standard = data_final_refined.rename(columns=rename_dict)
data_standard = data_standard[list(v for k,v in rename_dict.items())]
data_standard.columns

Index(['Crop', 'Season_id', 'Geom_id', 'Provider', 'Src_id', 'Src_File',
       'Crop_Group', 'Crop_Season', 'Crop_Variation', 'geometry', 'Sow_Date',
       'Est_Sow_Year', 'Sow_Day', 'Sow_Month', 'Sow_Year', 'Yield'],
      dtype='object')

In [39]:
data_standard.head()

,Crop,Season_id,Geom_id,Provider,Src_id,Src_File,Crop_Group,Crop_Season,Crop_Variation,geometry,Sow_Date,Est_Sow_Year,Sow_Day,Sow_Month,Sow_Year,Yield
0,Grassland,40d74a3f67010c-2022_EST,40d74a3f67010c,Kernel,0,data/2023_kernel.parquet + data/2023_kernel_ta...,Non Crop,None,None,"POLYGON ((33.20118 49.19568, 33.20114 49.19569...",NaT,2022.0,<NA>,<NA>,<NA>,NaN
1,Grassland,40d6888749dff4-2022_EST,40d6888749dff4,Kernel,1,data/2023_kernel.parquet + data/2023_kernel_ta...,Non Crop,None,None,"POLYGON ((32.61933 49.99669, 32.61915 49.99669...",NaT,2022.0,<NA>,<NA>,<NA>,NaN
2,Grassland,40d5c94c4f5fbc-2023-04-17,40d5c94c4f5fbc,Kernel,10,data/2023_kernel.parquet + data/2023_kernel_ta...,Non Crop,None,None,"POLYGON ((32.17066 50.28364, 32.17068 50.28369...",2023-04-17,NaN,17,4,2023,NaN
3,Soybean,40d60d3023aa1c-2023-05-14,40d60d3023aa1c,Kernel,1000,data/2023_kernel.parquet + data/2023_kernel_ta...,Legumes/Pulses,None,None,"MULTIPOLYGON (((32.9605 50.31382, 32.9603 50.3...",2023-05-14,NaN,14,5,2023,NaN
4,Soybean,40d60d359155cc-2023-05-12,40d60d359155cc,Kernel,1001,data/2023_kernel.parquet + data/2023_kernel_ta...,Legumes/Pulses,None,None,"POLYGON ((32.96351 50.30416, 32.96315 50.30419...",2023-05-12,NaN,12,5,2023,NaN


In [ ]:
#removing yield_clean
#def create_simple_mapper(user_df, schema_grouped):
#    """
#    Creates a UI using the simplified category dictionary.
#    """
#    user_cols = ['(None)'] + sorted(list(user_df.columns))
#    mapping_widgets = {}
#    
#    accordion = widgets.Accordion(children=[])
#    titles = []
#    children = []
#    
#    # Iterate directly through your simplified dictionary
#    for category, fields in schema_grouped.items():
#        titles.append(category)
#        rows = []
#        
#        for field in fields:
#            # Auto-Match Logic
#            default_val = '(None)'
#            for col in user_df.columns:
#                if col.lower() == field.lower():
#                    default_val = col
#                    break
#            
#            # UI Layout
#            lbl = widgets.Label(value=f"{field}:", layout=widgets.Layout(width='200px'))
#            dd = widgets.Dropdown(options=user_cols, value=default_val, layout=widgets.Layout(width='300px'))
#            
#            mapping_widgets[field] = dd
#            rows.append(widgets.HBox([lbl, dd], layout=widgets.Layout(margin='2px')))
#        
#        # Add tab content (Vertical stack)
#        children.append(widgets.VBox(rows, layout=widgets.Layout(padding='10px')))
#
#    accordion.children = tuple(children)
#    for i, title in enumerate(titles):
#        accordion.set_title(i, title)
#        
#    display(accordion)
#    return mapping_widgets
#
## --- RUN THE UI ---
## Pass the dataframe and the NEW generic dictionary
#widgets_dict = create_simple_mapper(data_final_refined, utils.target_schema_grouped)
#
## --- BUTTON LOGIC WITH "UNASSIGNED" CHECK ---
#btn = widgets.Button(description="Generate Mapping & Check Missing", layout=widgets.Layout(width='300px'))
#output = widgets.Output()
#
#def on_click(b):
#    with output:
#        output.clear_output()
#        final_mapping = {}
#        assigned_source_cols = set()
#        
#        # 1. Build the mapping dict
#        for field, widget in widgets_dict.items():
#            if widget.value != '(None)':
#                final_mapping[widget.value] = field
#                assigned_source_cols.add(widget.value)
#        
#        # 2. Find columns in the Input File that were NOT assigned
#        all_source_cols = set(data_final_refined.columns)
#        unmapped_cols = all_source_cols - assigned_source_cols
#        
#        # 3. Print Results
#        print(f"Successfully mapped {len(final_mapping)} columns.\n")
#        
#        print("--- MAPPING DICTIONARY ---")
#        pp = pprint.PrettyPrinter(indent=4)
#        pp.pprint(final_mapping)
#        
#        print("\n" + "="*40)
#        print(f"UNMAPPED COLUMNS ({len(unmapped_cols)})")
#        print("The following columns from your file were NOT assigned to any category:")
#        print("="*40)
#        
#        if unmapped_cols:
#            # Sort them so they are easy to read
#            for col in sorted(list(unmapped_cols)):
#                print(f" • {col}")
#        else:
#            print("Great! All columns from your file have been assigned.")
#
#btn.on_click(on_click)
#display(widgets.VBox([widgets.Label(""), btn, output]))